<a href="https://colab.research.google.com/github/DCODE-ARMY/GAN/blob/main/CWGAN_refined.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Fri Jul  8 08:44:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# try:
#   tpu_addr='grpc://' + os.environ['COLAB_TPU_ADDR'] #google remote procedure call (GRPC)
#   print(tpu_addr)
#   tpu=tf.distribute.cluster_resolver.TPUClusterResolver(tpu_addr)
#   tf.config.experimental_connect_to_cluster(tpu)
#   tf.tpu.experimental.initialize_tpu_system(tpu)
#   strategy=tf.distribute.experimental.TPUStrategy(tpu)
#   print('Running on Tpu',tpu.cluster_spec().as_dict()['worker'])
#   print('Number of accelerators',strategy.num_replicas_in_sync)

# except ValueError:
#   print('fuck')


In [ ]:
# import tensorflow as tf
# a=tf.keras.applications.inception_v3.InceptionV3(include_top=True,
#     weights='imagenet')
# a.summary()

In [ ]:
!cp '/content/drive/MyDrive/GAN GENERATED/DiffAug.py' /content


In [ ]:
pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#use centralStorageStrategy

import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense,Conv2D,BatchNormalization,MaxPooling2D, Dropout,Conv2DTranspose,InputLayer,Concatenate,LeakyReLU,Embedding,Reshape,AveragePooling2D
import cv2 
from google.colab.patches import cv2_imshow
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import time
from tqdm import tqdm
import tensorflow_addons as tfa #used for spectral normalaization
from sklearn.utils import shuffle
import PIL
import PIL.Image
from DiffAug import DiffAugment as df

# TF_GPU_ALLOCATOR=cuda_malloc_async

In [ ]:
# a=Embedding(5,16)(1)
# # a.numpy()
# a=np.expand_dims(a.numpy(),axis=0)
# print(a.shape)
# a=tf.keras.layers.RepeatVector(4096)(a)
# print(a.shape)
# a=Reshape((256,256,1))(a)
# print(a.shape)
# print(a.numpy)
# c=a.numpy()
# print(c[0,0,255])

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
os.listdir('/content/drive/MyDrive/scrapped data')

['headphone', 'mouse', 'laptop', 'keyboard', 'monitor']

In [ ]:
#total image 1228/32= 38 || 1228/16=76 || 1228/26=46
batch_size = 16
num_channels = 3
num_classes = 5
image_size = 256
latent_dim = 256
channel_axis=3
steps_per_epoc=38
epochh=1
divide_array=np.array([6])

policy = 'color,translation,cutout'# color,

# adaptive discriminator augmentation
max_translation = 0.125
max_rotation = 0.125
max_zoom = 0.25
# target_accuracy = 0.85
# integration_steps = 1000

In [ ]:
# if np.any(74%divide_array==0) :
#   print(True)

# 74%divide_array

In [ ]:

# # @tf.function 

# It does not support generator
# It cannot be used with funcs returing model_variables
# It excepts to return atleast one Tensor


In [ ]:

def gene(data_dir='/content/drive/MyDrive/scrapped data',batch_size=batch_size,resize=(image_size,image_size)):
  cat=['laptop', 'keyboard', 'mouse', 'headphone', 'monitor']

  img_path_list=[]
  labels=[]

  #getting path to all images
  for i in cat:
    path=os.path.join(data_dir,i)
    for j in os.listdir(path):
      img_path_list.append(os.path.join(path,j))

  img_path_list=shuffle(img_path_list)

  for i in img_path_list:
    # print(cat.index(i.split('/')[-2]))
    labels.append(cat.index(i.split('/')[-2]))


#converting labels to onehot encoded vectors  
  # labels=tf.keras.utils.to_categorical(labels,5)
 

  img_list=[]
  labels_list=[]
 
  for i,j in enumerate(img_path_list):
    # img=cv2.resize(cv2.imread(j,cv2.IMREAD_UNCHANGED),resize,interpolation=cv2.INTER_AREA)
    img=np.array(PIL.Image.open(j))
    # tensor = tf.image.resize(tensor, (128,128))
    # tensor = tf.subtract(tf.divide(tensor, 127.5), 1)

    img=img.astype(np.float32)
   
    img=tf.image.resize(img,[image_size,image_size],method=tf.image.ResizeMethod.NEAREST_NEIGHBOR,antialias=False)
    # img=img.astype(np.float32)
    # img=(img-127.5)/127.5
    img = tf.subtract(tf.divide(img, 127.5), 1)
    assert not np.any(np.isnan(img))
    img_list.append(img)
    
    labels_list.append(labels[i])

    if len(img_list)==batch_size:
      
      data=np.array(img_list,dtype=np.float32),np.expand_dims(np.array(labels_list,dtype=np.int8),axis=1)#not necessarily convert to tensor
      yield data
      img_list.clear()
      labels_list.clear()



#Tensorspec can be used to declare numpy signature but with dtype specified in tf

dataset=tf.data.Dataset.from_generator(gene,output_signature=(tf.TensorSpec(shape=(batch_size, image_size, image_size, channel_axis),dtype=tf.dtypes.float32),tf.TensorSpec(shape=(batch_size,1),dtype=tf.dtypes.int8)))
dataset=dataset.prefetch(buffer_size=tf.data.AUTOTUNE).repeat(1000)

In [ ]:
#prepping  dataset for Tpu 
# strategy.experimental_

In [ ]:
# a=36
# if np.any(35%np.array([36,37,38])==0):
#   print(True)
# else:
#   print(False)

In [ ]:
class KID(keras.metrics.Metric):
  def __init__(self, name="kid", **kwargs):
    super().__init__(name=name, **kwargs)

    # KID is estimated per batch and is averaged across batches
    self.kid_tracker = keras.metrics.Mean()

    # a pretrained InceptionV3 is used without its classification layer
    # transform the pixel values to the 0-255 range, then use the same
    # preprocessing as during pretraining
    self.encoder = keras.Sequential(
        [
            layers.InputLayer(input_shape=(image_size, image_size, 3)),
            layers.Rescaling(255.0),
            layers.Resizing(height=kid_image_size, width=kid_image_size),
            layers.Lambda(keras.applications.inception_v3.preprocess_input),
            keras.applications.InceptionV3(
                include_top=False,
                input_shape=(kid_image_size, kid_image_size, 3),
                weights="imagenet",
            ),
            layers.GlobalAveragePooling2D(),
        ],
        name="inception_encoder",
    )

  def polynomial_kernel(self, features_1, features_2):
    feature_dimensions = tf.cast(tf.shape(features_1)[1], dtype=tf.float32)
    return (features_1 @ tf.transpose(features_2) / feature_dimensions + 1.0) ** 3.0

  def update_state(self, real_images, generated_images, sample_weight=None):
    real_features = self.encoder(real_images, training=False)
    generated_features = self.encoder(generated_images, training=False)

    # compute polynomial kernels using the two sets of features
    kernel_real = self.polynomial_kernel(real_features, real_features)
    kernel_generated = self.polynomial_kernel(
        generated_features, generated_features
    )
    kernel_cross = self.polynomial_kernel(real_features, generated_features)

    # estimate the squared maximum mean discrepancy using the average kernel values
    batch_size = tf.shape(real_features)[0]
    batch_size_f = tf.cast(batch_size, dtype=tf.float32)
    mean_kernel_real = tf.reduce_sum(kernel_real * (1.0 - tf.eye(batch_size))) / (
        batch_size_f * (batch_size_f - 1.0)
    )
    mean_kernel_generated = tf.reduce_sum(
        kernel_generated * (1.0 - tf.eye(batch_size))
    ) / (batch_size_f * (batch_size_f - 1.0))
    mean_kernel_cross = tf.reduce_mean(kernel_cross)
    kid = mean_kernel_real + mean_kernel_generated - 2.0 * mean_kernel_cross

    # update the average KID estimate
    self.kid_tracker.update_state(kid)

  def result(self):
    return self.kid_tracker.result()

  def reset_state(self):
    self.kid_tracker.reset_state()


In [ ]:
def conv2d_bn(x,filter,kernel_size,stride,padding='same',activation=tf.keras.layers.LeakyReLU(alpha=0.2),is_initializer=True,is_regularizer=False,initializer= tf.keras.initializers.RandomNormal()
,regularizer=None,name=None):
  if name is not None:
    bn_name = name + '_bn'
    conv_name = name + '_conv'
  else:
    bn_name = None
    conv_name = None

  


  if is_initializer and is_regularizer:
    
    x=tfa.layers.SpectralNormalization(layers.Conv2D(filters=filter,kernel_size=kernel_size,strides=stride,padding=padding,activation=activation,kernel_initializer=initializer,regularizer=regularizer,name=conv_name))(x)

  elif is_regularizer:
    
    x=tfa.layers.SpectralNormalization(layers.Conv2D(filters=filter,kernel_size=kernel_size,strides=stride,padding=padding,activation=activation,regularizer=regularizer,name=conv_name))(x)

  elif is_initializer:
    
    x=tfa.layers.SpectralNormalization(layers.Conv2D(filters=filter,kernel_size=kernel_size,strides=stride,padding=padding,activation=activation,kernel_initializer=initializer,name=conv_name))(x)

  else:
    x=tfa.layers.SpectralNormalization(layers.Conv2D(filters=filter,kernel_size=kernel_size,strides=stride,padding=padding,activation=activation,name=conv_name))(x)
   
  

  
  return x

    

    


In [ ]:

# a=np.random.rand(0,256)
# # a=np.expand_dims(np.random.rand(None,256),axis=1)
# print(a.shape)
# a=tf.keras.layers.RepeatVector(256)(a)
# tf.print(a.numpy())

In [ ]:
# embedding=Embedding(num_classes,10)

# for i in range(10):
#   q=embedding()
#   print(q.numpy(),q.numpy().shape)


In [ ]:
# tf.config.run_functions_eagerly(True)
# embedding=Embedding(num_classes,50)


def discriminator():
  # initializer = tf.keras.initializers.RandomNormal(mean=0., stddev=.05)

  # Create the discriminator.
  inp =layers.Input(shape=[image_size, image_size,channel_axis])

  label_inp_layer=layers.Input(shape=(1,))
  label_inp=Embedding(num_classes,16)(label_inp_layer)

  label_inp=tfa.layers.SpectralNormalization(layers.Dense(16,activation=tf.keras.layers.LeakyReLU(alpha=0.2) )) (label_inp)

  label_inp=layers.Flatten()(label_inp)
 
  label_inp=tf.keras.layers.RepeatVector(4096)(label_inp) #try using 16 embedding
  
  
  # label_inp= tf.repeat(label_inp.numpy(), repeats=[image_size * image_size])

  
  label_inp=Reshape((image_size,image_size,1))(label_inp)

  
  x = Concatenate(axis=-1)([inp, label_inp])

  #filter,kernel_size,stride,padding,activation,initializer,regularizer,dropout_val

  # x=conv2d_bn(x,64,3,1,padding='same')

  # batch_size,128,128,32
  x=conv2d_bn(x,64,3,2,padding='same')


  x=conv2d_bn(x,32,1,1,padding='valid')

  x=conv2d_bn(x,192,3,1,padding='valid')
  x=layers.Dropout(0.2)(x)

  x=conv2d_bn(x,80,1,1,padding='valid')

  # batch_size,128,128,64
  x=conv2d_bn(x,64,3,1,padding='valid') #maxpool_2

  x=conv2d_bn(x,32,1,1,padding='valid')

  x=conv2d_bn(x,64,5,1,padding='same')
  # x=layers.Dropout(0.4)(x)

  x=conv2d_bn(x,80,1,1,padding='valid')

  x=conv2d_bn(x,64,3,2,padding='same')

  



  
  
  #block 1
  #batch_size,35 x 35 x 256
 
  branch1x1=conv2d_bn(x,64,1,1)

  branch5x5=conv2d_bn(x, 48, 1, 1)
  branch5x5 = conv2d_bn(branch5x5, 32, 5, 1)
  
  branch3x3dbl = conv2d_bn(x, 64, 1, 1)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 43, 5, 1)
  # branch3x3dbl=layers.Dropout(0.4)(branch3x3dbl)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 1)
  
  branch_pool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
  branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
  x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed0')

  x=conv2d_bn(x,128,1,1)
  x=conv2d_bn(x,196,3,2)

  x=layers.Dropout(0.2)(x)
  #block 2
  #batch_size,35 x 35 x 256
  branch1x1=conv2d_bn(x,64,1,1)

  branch5x5=conv2d_bn(x, 48, 1, 1)
  branch5x5 = conv2d_bn(branch5x5, 64, 5, 1)

  branch3x3dbl = conv2d_bn(x, 64, 1, 1)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 1)
  # branch3x3dbl=layers.Dropout(0.2)(branch3x3dbl)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 5, 1)
  # branch3x3dbl=layers.Dropout(0.2)(branch3x3dbl)
 
  branch_pool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
  branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
  x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed1')

  x=layers.Dropout(0.2)(x)

  # x=conv2d_bn(x, 64, 5, 2)

  #block 3
  #batch_size,35 x 35 x 256
  branch1x1=conv2d_bn(x,64,1,1)

  branch5x5=conv2d_bn(x, 48, 1, 1)
  branch5x5 = conv2d_bn(branch5x5, 32, 5, 1)

  branch3x3dbl = conv2d_bn(x, 64, 1, 1)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 1)
  # branch3x3dbl=layers.Dropout(0.4)(branch3x3dbl)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 64, 3, 1)

  branch_pool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
  branch_pool = conv2d_bn(branch_pool, 32, 1, 1)
  x = layers.concatenate(
        [branch1x1, branch5x5, branch3x3dbl, branch_pool],
        axis=channel_axis,
        name='mixed2')


  
  x=layers.Dropout(0.2)(x)
  #block 3
  #batch_size,17 x 17 x 768
  branch3x3 = conv2d_bn(x, 196, 3, 2,padding='valid')

  branch3x3dbl = conv2d_bn(x, 64, 1, 1)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 1,)
  # branch3x3dbl=layers.Dropout(0.4)(branch3x3dbl)
  branch3x3dbl = conv2d_bn(branch3x3dbl, 96, 3, 2,padding='valid')
  
  branch_pool = MaxPooling2D((3, 3), strides=(2, 2))(x)

  x = layers.concatenate([branch3x3, branch3x3dbl,branch_pool], axis=channel_axis, name='mixed3')
  # x=layers.Dropout(0.4)(x)

  # mixed 4: 17 x 17 x 768
  branch1x1 = conv2d_bn(x, 192, 1, 1)

  branch7x7 = conv2d_bn(x, 128, 1, 1)
  branch7x7 = conv2d_bn(branch7x7, 128, (1,7), 1)
  # branch7x7=layers.Dropout(0.2)(branch7x7)
  branch7x7 = conv2d_bn(branch7x7, 128, (7,1), 1)

  branch7x7dbl = conv2d_bn(x, 128, 1, 1)
  branch7x7dbl = conv2d_bn(branch7x7dbl, 128, (7,1), 1)
  # branch7x7dbl=layers.Dropout(0.2)(branch7x7dbl)
  branch7x7dbl = conv2d_bn(branch7x7dbl, 128, (1,7), 1)
  # branch7x7dbl=layers.Dropout(0.2)(branch7x7dbl)
  branch7x7dbl = conv2d_bn(branch7x7dbl, 128, (7,1), 1)
  # branch7x7dbl=layers.Dropout(0.2)(branch7x7dbl)
  branch7x7dbl = conv2d_bn(branch7x7dbl, 128, (1,7), 1)

  branch_pool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
  branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
  x = layers.concatenate([branch1x1, branch7x7, branch7x7dbl, branch_pool],axis=channel_axis,name='mixed4')

  # x=layers.Dropout(0.4)(x)


  x=layers.Dropout(0.2)(x)
  # mixed 5,6  batch_size,17 x 17 x 768
  for i in range(2):
    branch1x1 = conv2d_bn(x, 192, 1, 1)

    branch7x7 = conv2d_bn(x, 160, 1, 1)
    branch7x7 = conv2d_bn(branch7x7, 128, (1,7), 1)
    branch7x7 = conv2d_bn(branch7x7, 128, (7,1), 1)

    branch7x7dbl = conv2d_bn(x, 140, 1, 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, (7,1), 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, (1,7), 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, (7,1), 1)
    branch7x7dbl = conv2d_bn(branch7x7dbl, 128, (1,7), 1)

    branch_pool =AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)

    branch_pool = conv2d_bn(branch_pool, 128, 1, 1)
    x = layers.concatenate(
        [branch1x1, branch7x7, branch7x7dbl, branch_pool],
        axis=channel_axis,
        name='mixed' + str(5 + i))
    if i==0:
      x=layers.Dropout(0.4)(x)


  #  # mixed 7: 17 x 17 x 768
  # branch1x1 = conv2d_bn(x, 192, 1, 1)

  # branch7x7 = conv2d_bn(x, 192, 1, 1)
  # branch7x7 = conv2d_bn(branch7x7, 192, (1,7), 1)
  # branch7x7 = conv2d_bn(branch7x7, 192, (7,1), 1)

  # branch7x7dbl = conv2d_bn(x, 192, 1, 1)
  # branch7x7dbl = conv2d_bn(branch7x7dbl, 192, (1,7), 1)
  # branch7x7dbl = conv2d_bn(branch7x7dbl, 192, (7,1), 1)
  # branch7x7dbl = conv2d_bn(branch7x7dbl, 192, (1,7), 1)
  # branch7x7dbl = conv2d_bn(branch7x7dbl, 192, (7,1), 1)

  # branch_pool = AveragePooling2D((3, 3), strides=(1, 1), padding='same')(x)
  # branch_pool = conv2d_bn(branch_pool, 192, 1, 1)
  # x = layers.concatenate(
  #     [branch1x1, branch7x7, branch7x7dbl, branch_pool],
  #     axis=channel_axis,
  #     name='mixed7')
  
  # x=layers.Dropout(0.2)(x)
  
  x=conv2d_bn(x, 192, 1, 1,padding='same')

  x=conv2d_bn(x, 64, 3, 1,padding='valid')
  x=layers.Dropout(0.2)(x)
 
  # x=conv2d_bn(x, 192, 3, 1,padding='same')
  # # x=layers.Dropout(0.2)(x)

  # x=conv2d_bn(x, 64, 1, 1,padding='valid')
  # # x=layers.Dropout(0.2)(x)

  x=conv2d_bn(x, 64, 5, 2,padding='same')
  # x=layers.Dropout(0.2)(x)

  x=conv2d_bn(x, 32, 1, 1,padding='valid')
  x=conv2d_bn(x, 64, 5, 2,padding='same')
  # x=layers.Dropout(0.2)(x)

  x=conv2d_bn(x, 128, 3, 2,padding='valid')
  # x=conv2d_bn(x, 32, 1, 1,padding='valid')
  # x=conv2d_bn(x, 64, 2, 1,padding='valid')
  # x=layers.Dropout(0.3)(x)
  
  x=layers.Flatten()(x)

  # x=tfa.layers.SpectralNormalization(layers.Dense(units=1024,use_bias=False,kernel_regularizer=tf.keras.regularizers.L2(0.1),kernel_initializer='random_normal'))(x)
  # x=layers.Activation('LeakyReLU')(x)
  x=layers.Dropout(0.5)(x)

  x=layers.Dense(units=1)(x)

  return keras.Model(inputs=[label_inp_layer,inp], outputs=x)

discriminator=discriminator()




discriminator.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 16)        80          ['input_2[0][0]']                
                                                                                                  
 spectral_normalization (Spectr  (None, 1, 16)       288         ['embedding[0][0]']              
 alNormalization)                                                                                 
                                                                                                  
 flatten (Flatten)              (None, 16)           0           ['spectral_normalization[0][0

In [ ]:
# discriminator.summary()


In [ ]:
def generator():

  # kernel_regularizer=regularizers.L2(1e-4)
  
  initializer =tf.keras.initializers.RandomNormal()

  label_inp_layer=layers.Input(shape=(1,))
  x=Embedding(num_classes,16)(label_inp_layer)
  # x=Dense(1*1*256)(x)
  # x=layers.LeakyReLU(alpha=0.2)(x)
  x=Reshape((1,1,16))(x)

  inp =layers.Input(shape=(latent_dim,))
  # y=layers.Dense(1 * 1 * latent_dim)(inp)
  # y=layers.LeakyReLU(alpha=0.2)(y)
  y=layers.Reshape((1, 1, latent_dim))(inp)
  
  y=Concatenate(axis=-1)([y, x])

  y=Dense(256)(y)
  y=layers.LeakyReLU(alpha=0.2)(y)
 

  #(batch_size, 2, 2,514)
  y= layers.Conv2DTranspose(200,4,strides=2,padding='same',kernel_initializer= initializer)(y)
  y=layers.BatchNormalization()(y)
  y=layers.LeakyReLU()(y)
  y=layers.Dropout(.5)(y)

  #(batch_size, 4, 4,514)
  y= layers.Conv2DTranspose(200,4,strides=2,padding='same',kernel_initializer= initializer)(y)
  y=layers.BatchNormalization()(y)
  y=layers.LeakyReLU()(y)
  y=layers.Dropout(.3)(y)

  #(batch_size, 8, 8,514)
  y= layers.Conv2DTranspose(200,4,strides=2,padding='same',kernel_initializer= initializer)(y)
  y=layers.BatchNormalization()(y)
  y=layers.LeakyReLU()(y)
  y=layers.Dropout(.5)(y)

  #(batch_size, 16, 16,514) 50=128
  y= layers.Conv2DTranspose(200,4,strides=2,padding='same',kernel_initializer= initializer)(y)
  y=layers.BatchNormalization()(y)
  y=layers.LeakyReLU()(y)
  y=layers.Dropout(.3)(y)


  #(batch_size, 32, 32,514)
  y= layers.Conv2DTranspose(200,4,strides=2,padding='same',kernel_initializer=initializer )(y)#tf.random_normal_initializer(0., 0.05)
  y=layers.BatchNormalization()(y)
  y=layers.LeakyReLU()(y)
  y=layers.Dropout(.3)(y)


  #(batch_size, 64, 64,514)
  y= layers.Conv2DTranspose(200,4,strides=2,padding='same',kernel_initializer= initializer)(y)#,kernel_regularizer=regularizers.L2(1e-4)
  y=layers.BatchNormalization()(y)
  y=layers.LeakyReLU()(y)
  y=layers.Dropout(.3)(y)

  #(batch_size, 128, 128,514)
  y= layers.Conv2DTranspose(200,4,strides=2,padding='same',kernel_initializer= initializer)(y)
  y=layers.BatchNormalization()(y)
  y=layers.LeakyReLU()(y)
  y=layers.Dropout(.3)(y)
  
  #(batch_size, 256, 256,3)
  y=layers.Conv2DTranspose(3,4,strides=2,padding='same',kernel_initializer= initializer,activation='tanh')(y)

  return keras.Model(inputs=[label_inp_layer,inp], outputs=y)

generator=generator()

In [ ]:
# tf.keras.utils.plot_model(generator, show_shapes=True, dpi=64)
generator.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 256)]        0           []                               
                                                                                                  
 embedding_1 (Embedding)        (None, 1, 16)        80          ['input_3[0][0]']                
                                                                                                  
 reshape_2 (Reshape)            (None, 1, 1, 256)    0           ['input_4[0][0]']                
                                                                                            

In [ ]:
max_translation = 0.145 #125
max_rotation = 0.145 #125
max_zoom = 0.25 #0.25

In [ ]:
class AdaptiveAugmenter(keras.Model):
    def __init__(self):
        super().__init__()
        

        # stores the current probability of an image being augmented
        # self.probability = tf.Variable(0.0)

        # the corresponding augmentation names from the paper are shown above each layer
        # the authors show (see figure 4), that the blitting and geometric augmentations
        # are the most helpful in the low-data regime
        self.augmenter = keras.Sequential(
            [
                layers.InputLayer(input_shape=(image_size, image_size, 3)),
                # blitting/x-flip:
                layers.RandomFlip("horizontal"),
                # blitting/integer translation:
                layers.RandomTranslation(
                    height_factor=max_translation,
                    width_factor=max_translation,
                    interpolation="nearest",
                ),
                # geometric/rotation:
                layers.RandomRotation(factor=max_rotation),
                # geometric/isotropic and anisotropic scaling:
                layers.RandomZoom(
                    height_factor=(-max_zoom, 0.0), width_factor=(-max_zoom, 0.0)
                ),
            ],
            name="adaptive_augmenter",
        )
    

    def call(self, images, epoch,training=True):

      augmented_images = self.augmenter(images)
      

      if np.any(epoch % divide_array==0):
        return augmented_images
      else:
        return images


          

In [ ]:
# class aug:
#   def __init__(self):
#     pass

#   def call(self,epoch,images):
#     if np.any(epoch % divide_array==0):
#       images=df(images,policy=policy)
#       return images
#     else:
#       return images




In [ ]:
import numpy as np
import tensorflow as tf
a=np.array([1,2,3,4,5])
# a*=10
print((sum(a)/5)*10)

30.0


In [ ]:

# tf.config.run_functions_eagerly(True)

class ConditionalGAN(tf.keras.Model):
  def __init__(self,discriminator, generator, latent_dim,critic_steps,gp_weight=10.0):
    super(ConditionalGAN, self).__init__()
    self.discriminator = discriminator
    self.generator = generator
    self.latent_dim = latent_dim
    self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
    self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")
    self.gp_weight=gp_weight
    self.critic_steps=critic_steps
    # self.augmenter = AdaptiveAugmenter()
    self.diff=aug
    
    
    
    
    

  @property
  def metrics(self):
    return [self.gen_loss_tracker, self.disc_loss_tracker]

  def compile(self, d_optimizer, g_optimizer, generator_loss,critic_loss):
    super(ConditionalGAN,self).compile()
    self.d_optimizer = d_optimizer
    self.g_optimizer = g_optimizer
    self.generator_loss = generator_loss
    self.critic_loss=critic_loss
   
     


  @tf.function
  def gradient_penalty(self,batch_size, real_images, fake_images,labels):

    epsilon = tf.random.normal([batch_size, 1, 1, 1], 0.0, 1.0)
    diff = fake_images - real_images
    interpolated = real_images + (epsilon * diff)

    

    # interpolated=DiffAugment(interpolated,policy=policy)
    #saving interpolated images
    # for i in range(len(interpolated)):
    #   img = interpolated[i].numpy()
    #   img = keras.preprocessing.image.array_to_img(img) 
    #   img.save('/content/drive/MyDrive/interpolated'+'/'+str(i)+'.png')


    #axis=-1 concats to channel 
    # interpolated=tf.concat([interpolated,labels2img],axis=-1)
   

    with tf.GradientTape() as gp_tape:

      #tf.GradientTape().watch() is used to record constant vals.we can perform some computation on the variables we are watching. 
      #The computation can be anything from cubing it, x**3, to passing it through a neural network

      gp_tape.watch(interpolated)

      # 1. Get the discriminator output for this interpolated image.

      pred = self.discriminator([labels,interpolated], training=True)

    grads = gp_tape.gradient(pred, [interpolated])[0]


    # print('shape= ',grads.numpy().shape)
    #what is gradient
   #The gradient of any line or curve tells us the rate of change of one variable with respect to another.

    #Gradient norm scaling involves changing the derivatives of the loss function to have a given vector norm when the L2 vector norm 
    #(sum of the squared values) of the gradient vector exceeds a threshold value.

    norm = tf.sqrt(1e-8 + tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))

    gp = tf.reduce_mean((norm - 1.0) ** 2)
    # print('norm= ',norm,' gp= ',gp)

    return gp
    
  @tf.function
  def train_step(self,data):
   
    real_images,labels=data
  
    real_images=self.diff(epoch=epochh,images=real_images)
    
    for i in range(self.critic_steps):
      random_vectors=tf.random.normal(shape=(batch_size, self.latent_dim))
      
      #training descriminator
      with tf.GradientTape() as tape:

        fake_images = self.generator([labels,random_vectors], training=True)

        fake_images=self.diff(epoch=epochh,images=fake_images)
     

        fake_logits=self.discriminator([labels,fake_images],training=True)
        # print(fake_logits)


        real_logits=self.discriminator([labels,real_images], training=True)

        d_cost = self.critic_loss(real_logits=real_logits, fake_logits=fake_logits)

        gp = self.gradient_penalty(batch_size, real_images, fake_images,labels=labels) #both real and fake image have the shape of (256,256,3)

        #while interplating it will give the image with size(32, 256, 256, 3) so add the vector image labels
        d_loss = d_cost + gp * self.gp_weight

        #testing without GP
        # d_loss = d_cost

         

        #wasserstein
        grads = tape.gradient(d_loss, self.discriminator.trainable_variables)
        self.d_optimizer.apply_gradients(zip(grads, self.discriminator.trainable_variables))

#||||GENERATOR|||||||||||||||||||||||GENERATOR|||||||||||||||||||||||||||GENERATOR|||||||||||||||||||||||||||||||||||GENERATOR||||||||||||||||||||||||||||GENERATOR||||||||||
#--------------------GENERATOR-----------------------------GENERATOR---------------------------------GENERATOR---------------------------GENERATOR---------------------------
#||||||GENERATOR||||||||||||||||||GENERATOR|||||||||||||||||||||||||||GENERATOR|||||||||||||||||||||||||||||||||||GENERATOR|||||||||||||||||||||||||GENERATOR||||||||||||||||

    # Sample random points in the latent space for training generator
    random_vectors= tf.random.normal(shape=(batch_size, self.latent_dim))
   
    
    #train generator
    with tf.GradientTape() as tape:

      fake_images=self.generator([labels,random_vectors],training=True)
      fake_images=self.diff(epoch=epochh,images=fake_images) #self.augmenter

      fake_logits=self.discriminator([labels,fake_images],training=True)
      g_loss = self.generator_loss(fake_logits) 

    grads = tape.gradient(g_loss, self.generator.trainable_variables)
    self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_variables))

    # Monitor loss.
    self.gen_loss_tracker.update_state(g_loss)
    self.disc_loss_tracker.update_state(d_loss)

  
    return {
        "g_loss": self.gen_loss_tracker.result(),
        "d_loss": self.disc_loss_tracker.result(),
    }

@tf.function
def aug(epoch,images):
  return df(images,policy=policy)

  # if np.any(epoch % divide_array==0):
  #   images=df(images,policy=policy)
  #   return images
  # else:
  #   return images

@tf.function
def critic_loss(real_logits, fake_logits):
    real_loss = tf.reduce_mean(real_logits)
    fake_loss = tf.reduce_mean(fake_logits)
    return fake_loss - real_loss

@tf.function
def generator_loss(fake_logits):
    return -tf.reduce_mean(fake_logits)

In [ ]:
LR = 0.0002 # WGAN-GP paper recommends lr of 0.0002
d_optimizer = keras.optimizers.Adam(learning_rate=LR, beta_1=0.5, beta_2=0.9) # UPDATE for WGAN-GP: use Adam instead of RMSProp
g_optimizer = keras.optimizers.Adam(learning_rate=LR, beta_1=0.5, beta_2=0.9) # UPDATE for WGAN-GP: use Adam instead of RMSProp

In [ ]:
checkpoint_dir = '/content/drive/MyDrive/GAN GENERATED/chk'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

checkpoint = tf.train.Checkpoint(generator_optimizer=g_optimizer,discriminator_optimizer=d_optimizer,
                                 generator=generator,discriminator=discriminator)

manager = tf.train.CheckpointManager(
    checkpoint, directory=checkpoint_dir, max_to_keep=3)

In [ ]:
#tf.config.run_functions_eagerly(True) #will make all invocations of tf.function run eagerly instead of running as a traced graph function.This can be useful for debugging. 

import time
%matplotlib inline
class GANMonitor(keras.callbacks.Callback):

  def on_epoch_begin(self, epoch, logs=None):
    global epochh
    epochh=epoch+1
    
  def on_epoch_end(self, epoch, logs=None):
  #   print('GAN MONITOR',logs.keys)
    if (epoch+1) % 15 == 0:
      manager.save()

    if (epoch+1) % 5 == 0:
      pth=['laptop', 'keyboard', 'mouse', 'headphone', 'monitor']
      path=['/content/drive/MyDrive/GAN GENERATED/'+i for i in pth]
      
      random_latent_vectors = tf.random.normal(shape=(num_classes, latent_dim))#(5,256)
      classes=[[0],[1],[2],[3],[4]]
      classes=np.array(classes,dtype=np.int8)  #(5, 1)


      
      generated_images = self.model.generator([classes,random_latent_vectors])
      generated_images = (generated_images*127.5)+127.5

      for i in range(num_classes):
        img = generated_images[i].numpy()
        img = keras.preprocessing.image.array_to_img(img) 
      
        img.save(path[i]+'/'+str(epoch+1)+'.png')
      
    
    

callback=GANMonitor()

In [ ]:
# #restoring checkpoint. verify using assert below

# checkpoint_dir=('/content/drive/MyDrive/GAN GENERATED/chk')

# #checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir)).assert_consumed()

# checkpoint.restore(manager.latest_checkpoint).assert_consumed()


In [ ]:
1228/32

38.375

In [ ]:
#https://github.com/keras-team/keras/blob/master/keras/engine/training.py/Model
#It is extracted as tf.keras.Model
#so ConditionalGan(tf.keras.Model) super(ConditioanlGan,self).__init__() means we are calling the init func in keras/engine/training.py/Model

In [ ]:
#total image 1228 

csv_logger=tf.keras.callbacks.CSVLogger('/content/drive/MyDrive/GAN GENERATED/log.csv', separator=",", append=True)

cond_gan = ConditionalGAN( discriminator=discriminator , generator=generator, latent_dim=latent_dim, critic_steps=5)

cond_gan.compile( d_optimizer=d_optimizer, g_optimizer=g_optimizer, generator_loss=generator_loss, critic_loss=critic_loss)




In [ ]:
cond_gan.fit(dataset, epochs=1000,steps_per_epoch=38,callbacks=[callback,csv_logger]) #aprox 1000 epochs trained

In [ ]:
# count=0
# for i in range(1,301):
#   print(i*divide_array)
#   if np.any(i%divide_array==0):
#     count+=1
# print(count,(count/300)*100)

In [ ]:
import numpy as np
import matplotlib as mp
import pandas as pd

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/GAN GENERATED/log.csv')
df.head(10)

In [ ]:

df.drop('epoch',inplace=True,axis=1)
df.columns

In [ ]:
from matplotlib import pyplot as plt
plt.plot(df['gen'])


plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['gen'], loc='upper left')
plt.show()


plt.plot(df['dis'])


plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['dis'], loc='upper left')
plt.show()





In [ ]:
a=df['epoch']
for i,j in enumerate(a[546:]):
  df['epoch'][544+i]=544.0+i
  

In [ ]:
df['epoch'][543:]

In [ ]:
df.to_csv('/content/drive/MyDrive/GAN GENERATED/log.csv')